In [1]:
import re

import numpy as np

from langchain.schema import BaseRetriever
from langchain.utilities import GoogleSerperAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter, DocumentCompressorPipeline
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.schema import Document

In [2]:
class SearchRetriever(BaseRetriever):
    def __init__(self, search):
        self.search = search
    
    def get_relevant_documents(self, query: str):
        snippets = self.search.get_snippets(query)
        return [Document(page_content=s) for s in snippets]
    
    async def aget_relevant_documents(self, query: str):
        raise NotImplemented
        
        
base_retriever = SearchRetriever(DuckDuckGoSearchAPIWrapper())
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=OpenAIEmbeddings())
relevant_filter = EmbeddingsFilter(embeddings=OpenAIEmbeddings(), similarity_threshold=0.7)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)
retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever, base_compressor=pipeline_compressor
)

In [3]:
from langchain.chains.flare.base import FlareChain

flare = FlareChain.from_llm(OpenAI(), retriever=retriever, verbose=True)
flare.run("Given me a very detailed explanation of the California Budget Act of 2022")



> Entering new FlareChain chain...
Prompt after formatting:
Respond to the user message using any relevant context. Once you're done responding return FINISHED.

USER INPUT: Given me a very detailed explanation of the California Budget Act of 2022
CONTEXT: 
RESPONSE: 
Prompt after formatting:
Respond to the user message using any relevant context. Once you're done responding return FINISHED.

USER INPUT: Given me a very detailed explanation of the California Budget Act of 2022
CONTEXT: 
RESPONSE: The Budget Act of 2022 includes a $9.5 billion General Fund for a one‑time tax refund payment, the Better for Families Refund, also known as the Middle Class Tax Refund. It also includes $300 million ongoing General Fund to improve public health infrastructure, with about $200 million earmarked for
Prompt after formatting:
Respond to the user message using any relevant context. Once you're done responding return FINISHED.

USER INPUT: Given me a very detailed explanation of the California Bu

'The Budget Act of 2022 includes a $9.5 billion General Fund for a one‑time tax refund payment, the Better for Families Refund, also known as the Middle Class Tax Refund. It also includes $300 million ongoing General Fund to improve public health infrastructure, with about $200 million earmarked forcommunity development projects, technical assistance, and a variety of programs and activities to fund air and pollution monitoring and low- and zero-emission technologies. In addition, the bill will extend critical rural Medicare programs, telehealth flexibilities and the Acute Hospital Care at Home program, and prevent the Statutory Pay-As-You-Go (PAYGO) deficits from 2020-21 and 2021-22 from being carried forward into 2022-23. .'

In [4]:
OpenAI()("Given me a very detailed explanation of the California Budget Act of 2022")

'\n\nThe California Budget Act of 2022 is a piece of legislation passed by the state of California in order to establish a budget for the fiscal year of 2022. This act sets the framework for how the state will allocate funds and resources in order to ensure that its citizens have access to essential services and programs.\n\nThe act includes several key components, the first of which is a total spending limit. This spending limit is determined by the state legislature and is implemented to ensure that the state is not overspending in any given fiscal year. Once the spending limit is set, the state legislature then allocates funds to various departments and programs.\n\nThe act also includes provisions that outline how much money each department and program receives. This allows the state to ensure that each department and program is receiving the necessary funds to operate effectively. Additionally, the act sets parameters for any new spending initiatives, such as tax increases or cuts